In [1]:
import os
import pandas as pd
import nibabel as nib
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Input
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split

def load_mris_and_labels(mri_directory, csv_file):
    df = pd.read_csv(csv_file)
    mris = []
    labels = []
    ids = []

    for filename in os.listdir(mri_directory):
        if filename.endswith(".nii") or filename.endswith(".nii.gz"):
            rsct_id = filename.split('_')[0].split('-')[1]  # Adjusted to match your filename format
            img = nib.load(os.path.join(mri_directory, filename))
            label = df.loc[df['record_id'] == rsct_id, 'surg_engel'].values[0]

            mris.append(img)
            labels.append(label)
            ids.append(rsct_id)

    return mris, labels, ids

def create_model(input_shape):
    """ Create a 3D CNN model. """
    inputs = Input(shape=input_shape)
    x = Conv3D(32, kernel_size=(3, 3, 3), activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

2024-06-16 20:52:49.232761: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 20:52:49.270290: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 20:52:49.962914: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the data
mris, labels, ids = load_mris_and_labels('../../scripts/flirt_raw', '../../data/processed/label_df.csv')

# Convert MRI data to numpy array
mris_array = np.array([mri.get_fdata() for mri in mris])[..., np.newaxis]

In [3]:
# Prepare the data
X = mris_array[:60]
y = np.array(labels)[:60]

# Convert labels to binary (1 if label is 1, else 0)
y = np.where(y == 1, 1, 0)

# Verify labels for the subset
#print(f'Labels for the first 20 samples: {y}')

# Define input shape for the model
input_shape = X.shape[1:]

# Create the model
model = create_model(input_shape)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure the input data shape is correct
#print(f'X_train shape: {X_train.shape}')  # This should print (number of samples, 91, 109, 91, 1)
#print(f'y_train shape: {y_train.shape}')  # This should print (number of samples,)

# Fit the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=4)

2024-06-16 20:53:21.303442: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-16 20:53:21.331113: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-16 20:53:21.331165: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-16 20:53:21.334928: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-16 20:53:21.335000: I external/local_xla/xla/stream_executor

Epoch 1/10


I0000 00:00:1718535203.389619   34341 service.cc:145] XLA service 0x7fd02c005000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718535203.389676   34341 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-06-16 20:53:23.408548: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-16 20:53:23.541111: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 2/12 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.3750 - loss: 31368.1797

I0000 00:00:1718535206.931963   34341 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 108ms/step - accuracy: 0.4740 - loss: 27483.1543 - val_accuracy: 0.4167 - val_loss: 1482.7401
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.6338 - loss: 912.0750 - val_accuracy: 0.5833 - val_loss: 356.7805
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.9415 - loss: 28.9901 - val_accuracy: 0.5000 - val_loss: 161.9000
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 1.0000 - loss: 1.5237e-40 - val_accuracy: 0.6667 - val_loss: 59.9186
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 1.0000 - loss: 1.4013e-44 - val_accuracy: 0.8333 - val_loss: 48.7941
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 50.5259
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 51.0610
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 1.0000 - loss: 0.0000e+0

In [ ]:
# Save the model to a file
model.save('my_model.h5')

from tensorflow.keras.models import load_model

# Load the model from the file
#loaded_model = load_model('my_model.h5')


In [8]:
import matplotlib.pyplot as plt

# Make predictions on the test set
predictions = model.predict(X_test)

# Since the output is a probability, convert it to binary labels (0 or 1)
predicted_labels = (predictions > 0.5).astype(int)

# Select an example to visualize (e.g., the first example)
example_index = 0

# Get the MRI, true label, and prediction
mri_example = X_test[example_index]
true_label = y_test[example_index]
predicted_label = predicted_labels[example_index][0]

# Print the true label and prediction
print(f'True Label: {true_label}, Prediction: {predicted_label}')

# Plot the MRI slices
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Select three slices to visualize (middle slices of each dimension)
slices = [mri_example[mri_example.shape[0] // 2, :, :, 0],
          mri_example[:, mri_example.shape[1] // 2, :, 0],
          mri_example[:, :, mri_example.shape[2] // 2, 0]]

for i, slice in enumerate(slices):
    axes[i].imshow(slice.T, cmap="gray", origin="lower")
    axes[i].set_title(f'Slice {i+1}')

plt.suptitle(f'True Label: {true_label}, Prediction: {predicted_label}')
plt.show()

2024-06-16 20:56:17.588645: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.23GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-06-16 20:56:17.590547: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:580 : UNKNOWN: Failed to determine best cudnn convolution algorithm for:
%cudnn-conv-bias-activation.3 = (f32[12,32,89,107,89]{4,3,2,1,0}, u8[0]{0}) custom-call(f32[12,1,91,109,91]{4,3,2,1,0} %bitcast.158, f32[32,1,3,3,3]{4,3,2,1,0} %transpose.5, f32[32]{0} %arg2.3), window={size=3x3x3}, dim_labels=bf012_oi012->bf012, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv3D" op_name="functional_1_1/conv3d_1/convolution" source_file="/home/amaury/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1177}, backend_config=

UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/amaury/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/amaury/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/amaury/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/amaury/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/amaury/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, 

In [ ]:
import os
import pandas as pd
import nibabel as nib

def load_mris_and_labels(mri_directory, csv_file):
    # Load the CSV file
    df = pd.read_csv(csv_file)

    mris = []
    labels = []
    ids = []

    for filename in os.listdir(mri_directory):
        if filename.endswith(".nii") or filename.endswith(".nii.gz"):
            # Extract the RSCT id from the filename
            rsct_id = filename.split('_')[0].split('-')[1]  # Adjusted to match your filename format

            # Load the MRI file
            img = nib.load(os.path.join(mri_directory, filename))

            # Find the corresponding label
            label = df.loc[df['record_id'] == rsct_id, 'surg_engel'].values[0]

            mris.append(img)
            labels.append(label)
            ids.append(rsct_id)

    return mris, labels, ids

# Usage
mris, labels, ids = load_mris_and_labels('../../scripts/flirt_raw', '../../data/processed/label_df.csv')

# Print the shape, ID, and surg_engel for the first 10 MRIs
for i in range(10):
    print(f'ID: {ids[i]}, Shape: {mris[i].shape}, surg_engel: {labels[i]}')

# MRI preprocessing

In [ ]:
import os
import pandas as pd
from datetime import datetime

def find_latest_t1w_scan(base_path):
    participant_scans = {}
    for item in os.listdir(base_path):
        # Check if the folder name starts with 'sub-', indicating a participant folder
        if item.startswith('sub-'):
            participant_path = os.path.join(base_path, item)
            # Strip 'sub-' prefix to use as the dictionary key
            participant_id = item[4:]  # Removes the first four characters 'sub-'
            latest_date = None
            latest_file = None

            # Check each session folder within the participant directory
            for session_folder in os.listdir(participant_path):
                if session_folder.startswith('ses-'):
                    session_date = session_folder.split('-')[1]  # Extract the date from the session folder name
                    session_path = os.path.join(participant_path, session_folder)
                    anat_path = os.path.join(session_path, 'anat')  # 'anat' folder inside the session folder

                    if os.path.exists(anat_path) and os.path.isdir(anat_path):
                        # Iterate over all .nii files in the 'anat' directory
                        for file in os.listdir(anat_path):
                            if file.endswith('T1w.nii'):
                                file_date = datetime.strptime(session_date, "%Y%m%d")
                                # Update if this file's date is more recent
                                if latest_date is None or file_date > latest_date:
                                    latest_date = file_date
                                    latest_file = os.path.join(anat_path, file)

            if latest_file:
                participant_scans[participant_id] = latest_file

    return participant_scans

In [ ]:
base_path = '../../data/raw/resectMap_nifti_only_20240430'
latest_scans = find_latest_t1w_scan(base_path)
#latest_scans.items()

In [ ]:
# Load the prediction data
prediction_data = pd.read_csv('../../data/processed/label_df.csv', index_col=0)

# Convert latest_scans dictionary to DataFrame
scans_df = pd.DataFrame(list(latest_scans.items()), columns=['ParticipantID', 'ScanPath'])

# Merge the dataframes
prediction_data = prediction_data.rename(columns={"record_id" : "ParticipantID"})
final_data = prediction_data.merge(scans_df, on='ParticipantID', how='left')

In [ ]:
final_data.to_csv('../../data/processed/MRI_file_path.csv', index=False)

In [ ]:
import nibabel as nib

def load_mri(path):
    mri = nib.load(path)
    return mri.get_fdata()

In [ ]:
import numpy as np

def preprocess_mri(data):
    # Normalize the data to [0, 1]
    data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return data

In [ ]:
subset_df = final_data.dropna(subset=['ScanPath']).head(10)

In [ ]:
for _, row in subset_df.iterrows():
    mri_data = load_mri(row['ScanPath'])
    print(f"Participant ID: {row['ParticipantID']}, Shape: {mri_data.shape}")

In [ ]:
# Problem : MRIs with different shapes -> need resize
if False:
    X = []  # Image data
    y = []  # Labels

    # Remove participants without any T1w MRI scans
    final_data = final_data.dropna(subset=['ScanPath'])

    for _, row in subset_df.iterrows():
        mri_data = load_mri(row['ScanPath'])
        mri_data = preprocess_mri(mri_data)
        X.append(mri_data)
        y.append(row['surg_engel'])

    X = np.array(X)  # Convert list to array for training
    y = np.array(y)

In [ ]:
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

def resize_mri(data, new_shape=(64, 64, 64)):
    """ Resize the MRI to new_shape """
    # Calculate the zoom factors
    zoom_factors = np.array(new_shape) / np.array(data.shape)
    # Apply the zoom operation with bilinear interpolation
    return zoom(data, zoom_factors, order=1)  # order=1 (bilinear) is often a good trade-off


def preprocess_and_load_mris(df):
    X = []
    y = []

    for _, row in df.iterrows():
        mri_data = load_mri(row['ScanPath'])
        mri_data = preprocess_mri(mri_data)
        mri_data_resized = resize_mri(mri_data)
        X.append(mri_data_resized)
        y.append(row['surg_engel'])

    return np.array(X), np.array(y)

In [ ]:
X, y = preprocess_and_load_mris(final_data)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

def create_model(input_shape):
    """ Create a 3D CNN model. """
    model = Sequential([
        Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape),
        MaxPooling3D(pool_size=(2, 2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
X = X[..., np.newaxis]  # Add a channel dimension, assuming X doesn't already have it

In [ ]:
input_shape = X.shape[1:]
model = create_model(input_shape)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

model = Sequential([
    Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], X.shape[3], 1)),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=5)


In [ ]:
predictions = model.predict(X)

In [ ]:
from tensorflow.keras.utils import Sequence
import nibabel as nib
import numpy as np

class MRISequence(Sequence):
    def __init__(self, df, batch_size):
        self.df = df
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.df) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.df.iloc[idx * self.batch_size:(idx + 1) * self.batch_size]
        X = []
        y = []
        for _, row in batch_x.iterrows():
            mri_data = load_mri(row['ScanPath'])
            mri_data = preprocess_mri(mri_data)
            mri_data_resized = resize_mri(mri_data)
            X.append(mri_data_resized)
            y.append(row['Outcome'])
        
        return np.array(X), np.array(y)

# Usage
batch_size = 2  # You can adjust the batch size
train_gen = MRISequence(df=subset_df, batch_size=batch_size)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

def create_model(input_shape):
    # Create a Sequential model
    model = Sequential([
        Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape),
        MaxPooling3D(pool_size=(2, 2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Assuming your input shape from the preprocessed MRI data is known, e.g., (64, 64, 64, 1)
input_shape = (64, 64, 64, 1)
model = create_model(input_shape)


In [ ]:
model.fit(train_gen, epochs=10)


## Print system information

In [ ]:
import session_info

session_info.show()